In [1]:
!nvidia-smi

Fri May  7 04:34:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lucciffer","key":"98876e1a04292be5bd7cec8356ce9e60"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d soumikrakshit/anime-faces

 99% 435M/441M [00:13<00:00, 24.5MB/s]
100% 441M/441M [00:14<00:00, 32.7MB/s]


In [17]:
!unzip anime-faces.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/data/data/5499.png  
  inflating: data/data/data/55.png   
  inflating: data/data/data/550.png  
  inflating: data/data/data/5500.png  
  inflating: data/data/data/5501.png  
  inflating: data/data/data/5502.png  
  inflating: data/data/data/5503.png  
  inflating: data/data/data/5504.png  
  inflating: data/data/data/5505.png  
  inflating: data/data/data/5506.png  
  inflating: data/data/data/5507.png  
  inflating: data/data/data/5508.png  
  inflating: data/data/data/5509.png  
  inflating: data/data/data/551.png  
  inflating: data/data/data/5510.png  
  inflating: data/data/data/5511.png  
  inflating: data/data/data/5512.png  
  inflating: data/data/data/5513.png  
  inflating: data/data/data/5514.png  
  inflating: data/data/data/5515.png  
  inflating: data/data/data/5516.png  
  inflating: data/data/data/5517.png  
  inflating: data/data/data/5518.png  
  inflating: data/data/data/5519.png  
  inflating: dat

In [18]:
import os
import numpy as np
import cv2
from glob import glob
from matplotlib import pyplot
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [19]:
IMG_H = 64
IMG_W = 64
IMG_C = 3
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [20]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img)
    img = tf.image.resize_with_crop_or_pad(img, IMG_H, IMG_W)
    img = tf.cast(img, tf.float32)
    img = (img - 127.5) / 127.5
    return img

In [21]:
def tf_dataset(images_path, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(images_path)
    dataset = dataset.shuffle(buffer_size=10240)
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [22]:
def deconv_block(inputs, num_filters, kernel_size, strides, bn=True):
    x = Conv2DTranspose(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding="same",
        strides=strides,
        use_bias=False
        )(inputs)

    if bn:
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
    return x

In [24]:
def conv_block(inputs, num_filters, kernel_size, padding="same", strides=2, activation=True):
    x = Conv2D(
        filters=num_filters,
        kernel_size=kernel_size,
        kernel_initializer=w_init,
        padding=padding,
        strides=strides,
    )(inputs)

    if activation:
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.3)(x)
    return x

In [25]:
def build_generator(latent_dim):
    f = [2**i for i in range(5)][::-1]
    filters = 32
    output_strides = 16
    h_output = IMG_H // output_strides
    w_output = IMG_W // output_strides

    noise = Input(shape=(latent_dim,), name="generator_noise_input")

    x = Dense(f[0] * filters * h_output * w_output, use_bias=False)(noise)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((h_output, w_output, 16 * filters))(x)

    for i in range(1, 5):
        x = deconv_block(x,
            num_filters=f[i] * filters,
            kernel_size=5,
            strides=2,
            bn=True
        )

    x = conv_block(x,
        num_filters=3,
        kernel_size=5,
        strides=1,
        activation=False
    )
    fake_output = Activation("tanh")(x)

    return Model(noise, fake_output, name="generator")


In [26]:
def build_discriminator():
    f = [2**i for i in range(4)]
    image_input = Input(shape=(IMG_H, IMG_W, IMG_C))
    x = image_input
    filters = 64
    output_strides = 16
    h_output = IMG_H // output_strides
    w_output = IMG_W // output_strides

    for i in range(0, 4):
        x = conv_block(x, num_filters=f[i] * filters, kernel_size=5, strides=2)

    x = Flatten()(x)
    x = Dense(1)(x)

    return Model(image_input, x, name="discriminator")


In [27]:
class GAN(Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]

        for _ in range(2):
            ## Train the discriminator
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            generated_images = self.generator(random_latent_vectors)
            generated_labels = tf.zeros((batch_size, 1))

            with tf.GradientTape() as ftape:
                predictions = self.discriminator(generated_images)
                d1_loss = self.loss_fn(generated_labels, predictions)
            grads = ftape.gradient(d1_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

            ## Train the discriminator
            labels = tf.ones((batch_size, 1))

            with tf.GradientTape() as rtape:
                predictions = self.discriminator(real_images)
                d2_loss = self.loss_fn(labels, predictions)
            grads = rtape.gradient(d2_loss, self.discriminator.trainable_weights)
            self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        ## Train the generator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.ones((batch_size, 1))

        with tf.GradientTape() as gtape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = gtape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        return {"d1_loss": d1_loss, "d2_loss": d2_loss, "g_loss": g_loss}

In [30]:
def save_plot(examples, epoch, n):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        pyplot.subplot(n, n, i+1)
        pyplot.axis("off")
        pyplot.imshow(examples[i])
    filename = f"/content/samples/generated_plot_epoch-{epoch+1}.png"
    pyplot.savefig(filename)
    pyplot.close()

In [33]:
if __name__ == "__main__":
    ## Hyperparameters
    batch_size = 128
    latent_dim = 128
    num_epochs = 70
    images_path = glob("/content/data/data/data/*")

    d_model = build_discriminator()
    g_model = build_generator(latent_dim)

    # d_model.load_weights("saved_model/d_model.h5")
    # g_model.load_weights("saved_model/g_model.h5")

    d_model.summary()
    g_model.summary()

    gan = GAN(d_model, g_model, latent_dim)

    bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
    d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    gan.compile(d_optimizer, g_optimizer, bce_loss_fn)

    images_dataset = tf_dataset(images_path, batch_size)

    for epoch in range(num_epochs):
        gan.fit(images_dataset, epochs=1)
        g_model.save("/content/saved_model/g_model.h5")
        d_model.save("/content/saved_model/d_model.h5")

        n_samples = 25
        noise = np.random.normal(size=(n_samples, latent_dim))
        examples = g_model.predict(noise)
        save_plot(examples, epoch, int(np.sqrt(n_samples)))


Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 128)       204928    
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 128)     

In [36]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from matplotlib import pyplot

def save_plot(examples, n):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        pyplot.subplot(n, n, i+1)
        pyplot.axis("off")
        pyplot.imshow(examples[i])
    filename = "test_img.png"
    pyplot.savefig(filename)
    pyplot.close()

if __name__ == "__main__":
    model = load_model("/content/saved_model/g_model.h5")

    n_samples = 25     ## n should always be a square of an integer.
    latent_dim = 128
    latent_points = np.random.normal(size=(n_samples, latent_dim))
    examples = model.predict(latent_points)
    save_plot(examples, int(np.sqrt(n_samples)))

In [37]:
!zip -r samples /content

Streaming output truncated to the last 5000 lines.
  adding: content/data/data/data/4451.png (stored 0%)
  adding: content/data/data/data/19170.png (stored 0%)
  adding: content/data/data/data/13480.png (stored 0%)
  adding: content/data/data/data/19236.png (stored 0%)
  adding: content/data/data/data/11793.png (stored 0%)
  adding: content/data/data/data/8693.png (stored 0%)
  adding: content/data/data/data/11030.png (stored 0%)
  adding: content/data/data/data/2699.png (stored 0%)
  adding: content/data/data/data/166.png (stored 0%)
  adding: content/data/data/data/20573.png (stored 0%)
  adding: content/data/data/data/350.png (stored 0%)
  adding: content/data/data/data/3042.png (stored 0%)
  adding: content/data/data/data/3793.png (stored 0%)
  adding: content/data/data/data/12368.png (stored 0%)
  adding: content/data/data/data/10606.png (stored 0%)
  adding: content/data/data/data/3603.png (stored 0%)
  adding: content/data/data/data/18062.png (stored 0%)
  adding: content/data/d

In [40]:
!zip -r /content/samples  /content/samples/

  adding: content/samples/ (stored 0%)
  adding: content/samples/generated_plot_epoch-20.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-34.png (deflated 0%)
  adding: content/samples/generated_plot_epoch-32.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-65.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-55.png (deflated 0%)
  adding: content/samples/generated_plot_epoch-21.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-31.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-10.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-48.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-53.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-14.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-41.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-37.png (deflated 1%)
  adding: content/samples/generated_plot_epoch-68.png (deflated 0%)
  adding: